In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
import sys
import matplotlib.pyplot as plt
import seaborn as sns

module_path = (Path().resolve().parent/ "Modules")
sys.path.append(str(module_path))

pd.set_option("display.max_columns", None)
plt.rcParams["font.family"] = "IPAexGothic"

import my_modules, model_tuner, features # 自作モジュール
from skill_calculators import glicko2_calculator

%load_ext autoreload
%autoreload 2
%reload_ext autoreload

In [2]:
df = pd.read_csv("../Data/train_data_JV_2008.csv", encoding="cp932")

In [3]:
df = my_modules.preprocessing(df)
df = my_modules.common_process(df)

C:\Users\ken05\Documents\others\HORSE_RACING\Modules\my_modules.py:77: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df["place_num"] = df["place"].replace(place_dict).astype(int)


## EloRating moduleの試し打ち

In [4]:
df_train = df[(df.year >= 2020) & (df.year != 2025)]
df_test = df[df.year == 2025]

display(df_train.head(3))
display(df_test.head(3))

,race_id,year,month,day,times,place,daily,race_num,horse,jockey_id,trainer_id,owner,breeding_farm,breeding_place,horse_N,waku_num,horse_num,class_code,track_code,corner_num,dist,state,weather,age_code,sex,age,basis_weight,blinker,weight,inc_dec,weight_code,win_odds,win_mul_odds_Hi,win_mul_odds_Lo,rank,time_diff,time,corner1_rank,corner2_rank,corner3_rank,corner4_rank,last_3F_time,last_3F_rank,Ave_3F,PCI,PCI3,RPCI,last_3F_time_diff,leg,pop,prize,error_code,father,mother,broodmare_sire,broodmare_sire_type,horse_color,id,id_for_fold,field_type,flat_or_jump,turn_type,race_type,waku,datetime,age_type,dist_type,is_1st_rank,is_in_3rd_rank,target
254531,202001050601010108,2020,1,5,1,中山,1,1,イズジョーボヌール,1129,1043,泉一郎,森政巳,新ひだか町,16,4,8,7,24,2,1200,良,晴,12,牝,3,54.0,NaN,450.0,2.0,3,561.3,252.4,64.6,15,2.3,1156.0,NaN,NaN,12.0,13.0,40.3,12,35.3,37.6,40.57,36.0,1.4,後方,16.0,0,0,アサクサキングス,ビバロングライフ,コマンダーインチーフ,ニアークティック系,黒鹿,2017102716,2020010506010101,ダート,平地,R,中山ダート1200,inner,2020-01-05 01:02:00,3,splint,0,0,0
254530,202001050601010116,2020,1,5,1,中山,1,1,シラカワカツコ,1173,1031,大久保與志雄,新冠橋本牧場,新冠町,16,8,16,7,24,2,1200,良,晴,12,牝,3,51.0,NaN,404.0,4.0,3,233.1,83.6,21.6,14,2.1,1154.0,NaN,NaN,9.0,10.0,40.4,13,35.0,36.6,40.57,36.0,1.1,中団,13.0,0,0,ディープブリランテ,シャイニングピサ,Smart Strike,ネイティヴダンサー系,鹿毛,2017101747,2020010506010101,ダート,平地,R,中山ダート1200,outer,2020-01-05 01:02:00,3,splint,0,0,0
254529,202001050601010115,2020,1,5,1,中山,1,1,ラブカワールド,1177,1131,増田陽一,奥山博,日高町,16,8,15,7,24,2,1200,良,晴,12,牝,3,51.0,NaN,436.0,4.0,3,474.2,153.0,39.3,13,2.0,1153.0,NaN,NaN,15.0,16.0,39.9,9,35.4,38.7,40.57,36.0,1.5,後方,15.0,0,0,トゥザワールド,ラブカテリーナ,オレハマッテルゼ,ロイヤルチャージャー系,黒鹿,2017104350,2020010506010101,ダート,平地,R,中山ダート1200,outer,2020-01-05 01:02:00,3,splint,0,0,0


,race_id,year,month,day,times,place,daily,race_num,horse,jockey_id,trainer_id,owner,breeding_farm,breeding_place,horse_N,waku_num,horse_num,class_code,track_code,corner_num,dist,state,weather,age_code,sex,age,basis_weight,blinker,weight,inc_dec,weight_code,win_odds,win_mul_odds_Hi,win_mul_odds_Lo,rank,time_diff,time,corner1_rank,corner2_rank,corner3_rank,corner4_rank,last_3F_time,last_3F_rank,Ave_3F,PCI,PCI3,RPCI,last_3F_time_diff,leg,pop,prize,error_code,father,mother,broodmare_sire,broodmare_sire_type,horse_color,id,id_for_fold,field_type,flat_or_jump,turn_type,race_type,waku,datetime,age_type,dist_type,is_1st_rank,is_in_3rd_rank,target
23662,202501050601010107,2025,1,5,1,中山,1,1,コナベイション,1170,1023,ゴドルフィン,ダーレー・ジャパン・ファーム,日高町,16,4,7,7,24,2,1200,良,晴,12,牡,3,57.0,NaN,460.0,-4.0,3,11.6,3.5,1.9,9,1.6,1138.0,NaN,NaN,3.0,3.0,39.0,14,34.8,39.2,42.5,41.5,0.3,先行,5.0,0,0,タワーオブロンドン,ボーダレス,アドマイヤムーン,ネイティヴダンサー系,鹿毛,2022101692,2025010506010101,ダート,平地,R,中山ダート1200,inner,2025-01-05 01:02:00,3,splint,0,0,0
23667,202501050601010114,2025,1,5,1,中山,1,1,ノアファースト,1169,1076,佐山公男,清水牧場,平取町,16,7,14,7,24,2,1200,良,晴,12,牡,3,57.0,B,454.0,-10.0,3,189.5,39.2,18.9,14,2.8,1150.0,NaN,NaN,16.0,14.0,38.0,5,37.0,47.4,42.5,41.5,2.5,後方,12.0,0,0,カレンブラックヒル,ノアブリジット,サマーバード,ネイティヴダンサー系,栗毛,2022102350,2025010506010101,ダート,平地,R,中山ダート1200,outer,2025-01-05 01:02:00,3,splint,0,0,0
23666,202501050601010116,2025,1,5,1,中山,1,1,カツノトゥウェルヴ,1085,1031,勝野産業,天羽牧場,日高町,16,8,16,7,24,2,1200,良,晴,12,牝,3,55.0,NaN,470.0,-10.0,3,73.0,17.3,8.5,13,2.8,1150.0,NaN,NaN,14.0,14.0,38.3,11,36.7,45.8,42.5,41.5,2.2,後方,8.0,0,0,ヘニーヒューズ,ランスタン,ダイワメジャー,ロイヤルチャージャー系,栗毛,2022101055,2025010506010101,ダート,平地,R,中山ダート1200,outer,2025-01-05 01:02:00,3,splint,0,0,0


In [6]:
from skill_calculators import glicko2_calculator

horse_ts_calculator = glicko2_calculator(target_col="horse", prefix="horse")
df_train, feature_col = horse_ts_calculator.fit_transform(df_train, [])
display(df_train.head())
print(feature_col)

,race_id,year,month,day,times,place,daily,race_num,horse,jockey_id,trainer_id,owner,breeding_farm,breeding_place,horse_N,waku_num,horse_num,class_code,track_code,corner_num,dist,state,weather,age_code,sex,age,basis_weight,blinker,weight,inc_dec,weight_code,win_odds,win_mul_odds_Hi,win_mul_odds_Lo,rank,time_diff,time,corner1_rank,corner2_rank,corner3_rank,corner4_rank,last_3F_time,last_3F_rank,Ave_3F,PCI,PCI3,RPCI,last_3F_time_diff,leg,pop,prize,error_code,father,mother,broodmare_sire,broodmare_sire_type,horse_color,id,id_for_fold,field_type,flat_or_jump,turn_type,race_type,waku,datetime,age_type,dist_type,is_1st_rank,is_in_3rd_rank,target,horse_Glicko2,horse_Glicko2_RD,horse_Glicko2_min,horse_Glicko2_max,horse_Glicko2_after_racing
0,202001050601010108,2020,1,5,1,中山,1,1,イズジョーボヌール,1129,1043,泉一郎,森政巳,新ひだか町,16,4,8,7,24,2,1200,良,晴,12,牝,3,54.0,NaN,450.0,2.0,3,561.3,252.4,64.6,15,2.3,1156.0,NaN,NaN,12.0,13.0,40.3,12,35.3,37.6,40.57,36.0,1.4,後方,16.0,0,0,アサクサキングス,ビバロングライフ,コマンダーインチーフ,ニアークティック系,黒鹿,2017102716,2020010506010101,ダート,平地,R,中山ダート1200,inner,2020-01-05 01:02:00,3,splint,0,0,0,1500.0,250.0,750.0,2250.0,1182.188414
1,202001050601010111,2020,1,5,1,中山,1,1,コトブキノニ,1140,1035,江川伸夫,清水スタッド,様似町,16,6,11,7,24,2,1200,良,晴,12,牝,3,54.0,NaN,482.0,18.0,3,392.6,137.8,35.4,16,2.9,1162.0,NaN,NaN,9.0,11.0,41.2,16,35.0,35.0,40.57,36.0,1.1,後方,14.0,0,0,エイシンアポロン,ヤマホトトギス,サクラプレジデント,ロイヤルチャージャー系,鹿毛,2017105743,2020010506010101,ダート,平地,R,中山ダート1200,outer,2020-01-05 01:02:00,3,splint,0,0,0,1500.0,250.0,750.0,2250.0,1133.293889
2,202001050601010105,2020,1,5,1,中山,1,1,シンキングアイル,1179,1089,佐藤雄司,岡田スタツド,新ひだか町,16,3,5,7,24,2,1200,良,晴,12,牝,3,52.0,B,460.0,-2.0,3,82.2,26.6,7.1,8,1.1,1144.0,NaN,NaN,12.0,13.0,39.1,4,35.3,40.3,40.57,36.0,1.4,後方,10.0,0,0,アイルハヴアナザー,スピリテッドエアー,Danehill Dancer,ニアークティック系,栗毛,2017103287,2020010506010101,ダート,平地,R,中山ダート1200,inner,2020-01-05 01:02:00,3,splint,0,0,0,1500.0,250.0,750.0,2250.0,1524.446960
3,202001050601010107,2020,1,5,1,中山,1,1,オイデヤスダイジン,1119,1132,ニットー商事,斉藤安行,新冠町,16,4,7,7,24,2,1200,良,晴,12,牝,3,54.0,NaN,460.0,2.0,3,3.6,2.6,1.1,1,-0.0,1133.0,NaN,NaN,5.0,5.0,38.9,3,34.4,38.4,40.57,36.0,0.5,中団,2.0,510,0,ヘニーヒューズ,ハロータマ,ゼンノエルシド,ニアークティック系,鹿毛,2017101464,2020010506010101,ダート,平地,R,中山ダート1200,inner,2020-01-05 01:02:00,3,splint,1,1,1,1500.0,250.0,750.0,2250.0,1866.706111
4,202001050601010103,2020,1,5,1,中山,1,1,トーレスデルパイネ,726,1024,ミルファーム,山下恭茂,浦河町,16,2,3,7,24,2,1200,良,晴,12,牝,3,54.0,NaN,458.0,0.0,3,199.0,68.8,17.8,3,0.4,1137.0,NaN,NaN,9.0,9.0,38.7,2,35.0,40.4,40.57,36.0,1.1,中団,12.0,130,0,アンライバルド,ポーラーシェーン,Rock Hard Ten,ロイヤルチャージャー系,栗毛,2017103186,2020010506010101,ダート,平地,R,中山ダート1200,inner,2020-01-05 01:02:00,3,splint,0,1,3,1500.0,250.0,750.0,2250.0,1768.917226


['horse_Glicko2', 'horse_Glicko2_RD', 'horse_Glicko2_min', 'horse_Glicko2_max']


In [7]:
df_train[df_train.horse == "イクイノックス"]

,race_id,year,month,day,times,place,daily,race_num,horse,jockey_id,trainer_id,owner,breeding_farm,breeding_place,horse_N,waku_num,horse_num,class_code,track_code,corner_num,dist,state,weather,age_code,sex,age,basis_weight,blinker,weight,inc_dec,weight_code,win_odds,win_mul_odds_Hi,win_mul_odds_Lo,rank,time_diff,time,corner1_rank,corner2_rank,corner3_rank,corner4_rank,last_3F_time,last_3F_rank,Ave_3F,PCI,PCI3,RPCI,last_3F_time_diff,leg,pop,prize,error_code,father,mother,broodmare_sire,broodmare_sire_type,horse_color,id,id_for_fold,field_type,flat_or_jump,turn_type,race_type,waku,datetime,age_type,dist_type,is_1st_rank,is_in_3rd_rank,target,horse_Glicko2,horse_Glicko2_RD,horse_Glicko2_min,horse_Glicko2_max,horse_Glicko2_after_racing
77075,202108280404050502,2021,8,28,4,新潟,5,5,イクイノックス,5339,1051,シルクレーシング,ノーザンファーム,安平町,15,2,2,15,12,2,1800,良,曇,11,牡,2,54.0,NaN,474.0,NaN,3,4.6,2.1,1.4,1,-1.0,1474.0,NaN,NaN,3.0,3.0,34.5,1,36.45,55.7,54.53,54.8,0.2,先行,2.0,700,0,キタサンブラック,シャトーブランシュ,キングヘイロー,ニアークティック系,青鹿,2019105219,2021082804040505,芝,平地,L,新潟芝1800,inner,2021-08-28 05:06:00,2,mile,1,1,1,1500.000000,250.000000,750.000000,2250.000000,1862.225272
87670,202111200505051101,2021,11,20,5,東京,5,11,イクイノックス,5339,1126,シルクレーシング,ノーザンファーム,安平町,12,1,1,179,11,2,1800,良,晴,11,牡,2,55.0,NaN,482.0,8.0,3,2.6,1.6,1.3,1,-0.4,1462.0,NaN,9.0,10.0,8.0,32.9,1,36.65,61.4,59.30,54.8,1.4,中団,1.0,3800,0,キタサンブラック,シャトーブランシュ,キングヘイロー,ニアークティック系,青鹿,2019105219,2021112005050511,芝,平地,L,東京芝1800,inner,2021-11-20 11:01:00,2,mile,1,1,1,1862.225272,108.143851,1537.793719,2186.656825,2039.921565
106833,202204170603081118,2022,4,17,3,中山,8,11,イクイノックス,5339,1126,シルクレーシング,ノーザンファーム,安平町,18,8,18,195,17,4,2000,良,曇,12,牡,3,57.0,NaN,492.0,10.0,4,5.7,2.8,2.0,2,0.1,1598.0,7.0,5.0,4.0,3.0,34.6,8,36.51,55.5,57.17,54.1,0.4,先行,3.0,6000,0,キタサンブラック,シャトーブランシュ,キングヘイロー,ニアークティック系,青鹿,2019105219,2022041706030811,芝,平地,R,中山芝2000,outer,2022-04-17 11:02:00,3,intermediate,0,1,2,2039.921565,80.526271,1798.342752,2281.500379,2179.652318
112495,202205290502121118,2022,5,29,2,東京,C,11,イクイノックス,5339,1126,シルクレーシング,ノーザンファーム,安平町,18,8,18,195,11,4,2400,良,晴,12,牡,3,57.0,NaN,484.0,-8.0,4,3.8,1.8,1.3,2,0.0,2219.0,16.0,16.0,16.0,14.0,33.6,1,36.10,57.4,55.10,51.0,1.6,後方,2.0,8000,0,キタサンブラック,シャトーブランシュ,キングヘイロー,ニアークティック系,青鹿,2019105219,2022052905021211,芝,平地,L,東京芝2400,outer,2022-05-29 11:01:00,3,long,0,1,2,2179.652318,61.285696,1995.795229,2363.509407,2255.822126
130998,202210300504091107,2022,10,30,4,東京,9,11,イクイノックス,5339,1126,シルクレーシング,ノーザンファーム,安平町,15,4,7,195,11,3,2000,良,晴,13,牡,3,56.0,NaN,488.0,4.0,4,2.6,1.4,1.1,1,-0.1,1575.0,NaN,10.0,10.0,9.0,32.7,1,36.34,61.1,55.37,44.4,4.0,中団,1.0,20000,0,キタサンブラック,シャトーブランシュ,キングヘイロー,ニアークティック系,青鹿,2019105219,2022103005040911,芝,平地,L,東京芝2000,inner,2022-10-30 11:01:00,3,intermediate,1,1,1,2255.822126,56.602112,2086.015789,2425.628462,2315.922387
138225,202212250605081109,2022,12,25,5,中山,8,11,イクイノックス,5339,1126,シルクレーシング,ノーザンファーム,安平町,16,5,9,195,17,5,2500,良,晴,13,牡,3,55.0,NaN,492.0,4.0,4,2.3,1.4,1.1,1,-0.4,2324.0,8.0,9.0,6.0,3.0,35.4,2,36.95,54.4,54.57,52.5,0.5,先行,1.0,40000,0,キタサンブラック,シャトーブランシュ,キングヘイロー,ニアークティック系,青鹿,2019105219,2022122506050811,芝,平地,R,中山芝2500,outer,2022-12-25 11:02:00,3,long,1,1,1,2315.922387,51.735880,2160.714747,2471.130027,2363.202532
162080,202306250903081105,2023,6,25,3,阪神,8,11,イクイノックス,5339,1126,シルクレーシング,ノーザンファーム,安平町,17,3,5,195,17,4,2200,良,曇,13,牡,4,58.0,NaN,492.0,NaN,4,1.3,1.1,1.1,1,-0.0,2112.0,16.0,16.0,13.0,9.0,34.8,2,36.15,53.9,53.83,51.1,0.7,中団,1.0,22000,0,キタサンブラック,シャトーブランシュ,キングヘイロー,ニアークティック系,青鹿,2019105219,2023062509030811,芝,平地,R,阪神芝2200,inner,2023-06-25 11:03:00,over4,long,1,1,1,2363.202532,53.999410,2201.204303,2525.200762,2407.558012
177188,202310290504091107,2023,10,29,4,東京,9,11,イクイノックス,5339,1126,シルクレーシング,ノーザンファーム,安平町,11,6,7,195,11,3,2000,良,晴,13,牡,4,58.0,NaN,494.0,2.0,4,1.3,1.1,1.1,1,-0.4,1552.0,NaN,3.0,3.0,3.0,34.2,3,34.71,51.5,53.07,49.4,0.5,先行,1.0,22000,0,キタサンブラック,シャトーブランシュ,キングヘイロー,ニアークティック系,青鹿,2019105219,2023102905040911,芝,平地,L,東京芝2000,outer,2023-10-29 11:01:00,over4,intermediate,1,1,1,2407.

In [10]:
df_train.sort_values(by="horse_Glicko2", ascending=False).head(10)

,race_id,year,month,day,times,place,daily,race_num,horse,jockey_id,trainer_id,owner,breeding_farm,breeding_place,horse_N,waku_num,horse_num,class_code,track_code,corner_num,dist,state,weather,age_code,sex,age,basis_weight,blinker,weight,inc_dec,weight_code,win_odds,win_mul_odds_Hi,win_mul_odds_Lo,rank,time_diff,time,corner1_rank,corner2_rank,corner3_rank,corner4_rank,last_3F_time,last_3F_rank,Ave_3F,PCI,PCI3,RPCI,last_3F_time_diff,leg,pop,prize,error_code,father,mother,broodmare_sire,broodmare_sire_type,horse_color,id,id_for_fold,field_type,flat_or_jump,turn_type,race_type,waku,datetime,age_type,dist_type,is_1st_rank,is_in_3rd_rank,target,horse_Glicko2,horse_Glicko2_RD,horse_Glicko2_min,horse_Glicko2_max,horse_Glicko2_after_racing
219274,202409290604091114,2024,9,29,4,中山,9,11,ビクターザウィナー,5509,5676,Y.チュウ,Mr.A Sangster,豪,16,7,14,195,18,2,1200,良,曇,13,セ,6,58.0,NaN,494.0,1.0,4,12.5,4.1,2.8,6,0.4,1074.0,NaN,NaN,4.0,4.0,34.3,12,33.10,46.5,49.33,42.0,1.0,先行,6.0,0,0,Toronado,Noetic,Cape Cross,ニアークティック系,黒鹿,2018190001,2024092906040911,芝,平地,R,中山芝1200,outer,2024-09-29 11:02:00,over4,splint,0,0,0,3907.900436,173.195021,3388.315374,4427.485498,3050.234722
227301,202412010704021113,2024,12,1,4,中京,2,11,ミックファイア,5339,5667,KHレーシング,高橋フアーム,新ひだか,16,7,13,195,23,4,1800,良,晴,13,牡,4,58.0,NaN,500.0,-5.0,4,60.5,13.4,8.4,13,2.3,1524.0,15.0,15.0,16.0,15.0,37.9,10,37.25,48.3,50.97,49.2,1.3,後方,10.0,0,0,シニスターミニスター,マリアージュ,ブライアンズタイム,ロイヤルチャージャー系,鹿毛,2020104376,2024120107040211,ダート,平地,L,中京ダート1800,outer,2024-12-01 11:04:00,over4,mile,0,0,0,2929.025074,176.981105,2398.081759,3459.968388,1209.752614
191457,202402180501081112,2024,2,18,1,東京,8,11,スピーディキック,5404,5766,加藤鈴幸,熊谷武,浦河,16,6,12,195,23,2,1600,良,晴,14,牝,5,56.0,NaN,484.0,-3.0,4,107.0,19.9,12.7,13,2.0,1377.0,NaN,NaN,9.0,9.0,38.9,13,35.28,40.7,45.30,41.9,0.9,中団,15.0,0,0,タイセイレジェンド,デザートフラワー,サイレントディール,ロイヤルチャージャー系,栗毛,2019102512,2024021805010811,ダート,平地,L,東京ダート1600,outer,2024-02-18 11:01:00,over4,mile,0,0,0,2736.036906,161.634092,2251.134629,3220.939182,1493.284639
89812,202112050706021113,2021,12,5,6,中京,2,11,チュウワウィザード,5386,1058,中西忍,ノーザンファーム,安平町,16,7,13,195,23,4,1800,良,曇,13,牡,6,57.0,NaN,488.0,4.0,4,4.6,1.8,1.4,2,1.0,1507.0,10.0,10.0,11.0,11.0,36.2,2,37.25,52.9,52.63,52.4,0.8,後方,3.0,4000,0,キングカメハメハ,チュウワブロッサム,デュランダル,ロイヤルチャージャー系,青鹿,2015104879,2021120507060211,ダート,平地,L,中京ダート1800,outer,2021-12-05 11:04:00,over4,mile,0,1,2,2510.387903,127.299228,2128.490220,2892.285586,2505.575395
134593,202211270505081205,2022,11,27,5,東京,8,12,グランドグローリー,5464,5754,ウス牧場,Elevage Haras de Bourgeauville,英,18,3,5,195,11,4,2400,良,晴,13,牝,6,55.0,NaN,470.0,NaN,4,57.0,14.9,9.5,6,0.6,2243.0,8.0,10.0,12.0,12.0,34.2,5,36.70,57.3,58.43,56.7,0.6,中団,14.0,0,0,Olympic Glory,Madonna Lily,Daylami,ナスルーラ系,鹿毛,2016190001,2022112705050812,芝,平地,L,東京芝2400,inner,2022-11-27 12:01:00,over4,long,0,0,0,2476.768757,133.024905,2077.694042,2875.843472,2268.388806
180809,202311260505081202,2023,11,26,5,東京,8,12,イクイノックス,5339,1126,シルクレーシング,ノーザンファーム,安平町,18,1,2,195,11,4,2400,良,曇,13,牡,4,58.0,NaN,498.0,4.0,4,1.3,1.1,1.1,1,-0.7,2218.0,3.0,3.0,3.0,3.0,33.5,1,36.10,57.8,57.03,46.2,3.0,先行,1.0,50000,0,キタサンブラック,シャトーブランシュ,キングヘイロー,ニアークティック系,青鹿,2019105219,2023112605050812,芝,平地,L,東京芝2400,inner,2023-11-26 12:01:00,over4,long,1,1,1,2446.440047,50.287288,2295.578185,2597.301910,2477.570872
88887,202111280505081202,2021,11,28,5,東京,8,12,コントレイル,1014,1075,前田晋二,ノースヒルズ,新冠町,18,1,2,195,11,4,2400,良,晴,13,牡,4,57.0,NaN,456.0,-8.0,4,1.6,1.1,1.1,1,-0.3,2247.0,8.0,7.0,8.0,8.0,33.7,1,37.00,59.8,58.13,53.3,1.6,中団,1.0,30000,0,ディープインパクト,ロードクロサイト,Unbridled's Song,ネイティヴダンサー系,青鹿,2017101835,2021112805050812,芝,平地,L,東京芝2400,inner,2021-11-28 12:01:00,over4,long,1,1,1,2433.976909,64.804751,2239.562655,2628.391163,2457.845464
42878,202011290505091206,2020,11,29,5,東京,9,12,コントレイル,1014,1075,前田晋二,ノースヒルズ,新冠町,15,4,6,195,11,4,2400,良,曇,13,牡,3,55.0,NaN,456.0,-2.0,4,2.8,1.7,1.1,2,0.2,2232.0,9.0,9.0,9.0,9.0,34.3,1,36.30,55.8,55.07,42.8,3.7,中団,2.0,12000,0,ディープインパクト,ロードクロサイト,Unbridled's Song,ネイティヴダンサー系,青鹿,2017101835,2020112905050912,芝,平地,L,東京芝240

In [11]:
df_test_calced, feature_col_pred = horse_ts_calculator.transform(df_test, [])

In [14]:
df_train[df_train.horse == "クロワデュノール"]

,race_id,year,month,day,times,place,daily,race_num,horse,jockey_id,trainer_id,owner,breeding_farm,breeding_place,horse_N,waku_num,horse_num,class_code,track_code,corner_num,dist,state,weather,age_code,sex,age,basis_weight,blinker,weight,inc_dec,weight_code,win_odds,win_mul_odds_Hi,win_mul_odds_Lo,rank,time_diff,time,corner1_rank,corner2_rank,corner3_rank,corner4_rank,last_3F_time,last_3F_rank,Ave_3F,PCI,PCI3,RPCI,last_3F_time_diff,leg,pop,prize,error_code,father,mother,broodmare_sire,broodmare_sire_type,horse_color,id,id_for_fold,field_type,flat_or_jump,turn_type,race_type,waku,datetime,age_type,dist_type,is_1st_rank,is_in_3rd_rank,target,horse_Glicko2,horse_Glicko2_RD,horse_Glicko2_min,horse_Glicko2_max,horse_Glicko2_after_racing
205743,202406090503040506,2024,6,9,3,東京,4,5,クロワデュノール,1102,1151,サンデーレーシング,ノーザンファーム,安平町,11,6,6,15,11,2,1800,良,曇,11,牡,2,55.0,NaN,480.0,NaN,3,6.1,3.3,1.2,1,-0.4,1467.0,NaN,2.0,2.0,2.0,33.8,1,36.45,57.8,57.00,57.4,0.1,先行,3.0,720,0,キタサンブラック,ライジングクロス,Cape Cross,ニアークティック系,青鹿,2022105102,2024060905030405,芝,平地,L,東京芝1800,outer,2024-06-09 05:01:00,2,mile,1,1,1,1500.000000,250.000000,750.000000,2250.000000,1837.482950
225130,202411160505051104,2024,11,16,5,東京,5,11,クロワデュノール,1102,1151,サンデーレーシング,ノーザンファーム,安平町,9,4,4,179,11,2,1800,良,曇,11,牡,2,56.0,NaN,504.0,24.0,3,2.2,1.5,1.1,1,-0.1,1468.0,NaN,3.0,3.0,2.0,33.3,1,36.75,60.4,59.97,59.9,0.1,先行,1.0,3800,0,キタサンブラック,ライジングクロス,Cape Cross,ニアークティック系,青鹿,2022105102,2024111605050511,芝,平地,L,東京芝1800,inner,2024-11-16 11:01:00,2,mile,1,1,1,1837.482950,124.556506,1463.813433,2211.152466,2013.461303
230424,202412280605091106,2024,12,28,5,中山,9,11,クロワデュノール,1102,1151,サンデーレーシング,ノーザンファーム,安平町,18,3,6,195,17,4,2000,良,晴,11,牡,2,56.0,NaN,496.0,-8.0,3,1.8,1.3,1.1,1,-0.3,2005.0,7.0,7.0,4.0,3.0,34.9,2,36.69,55.1,54.07,52.6,0.6,先行,1.0,7000,0,キタサンブラック,ライジングクロス,Cape Cross,ニアークティック系,青鹿,2022105102,2024122806050911,芝,平地,R,中山芝2000,inner,2024-12-28 11:02:00,2,intermediate,1,1,1,2013.461303,91.892118,1737.784949,2289.137657,2143.086071


In [12]:
df_test_calced[df_test.horse == "クロワデュノール"]

,race_id,year,month,day,times,place,daily,race_num,horse,jockey_id,trainer_id,owner,breeding_farm,breeding_place,horse_N,waku_num,horse_num,class_code,track_code,corner_num,dist,state,weather,age_code,sex,age,basis_weight,blinker,weight,inc_dec,weight_code,win_odds,win_mul_odds_Hi,win_mul_odds_Lo,rank,time_diff,time,corner1_rank,corner2_rank,corner3_rank,corner4_rank,last_3F_time,last_3F_rank,Ave_3F,PCI,PCI3,RPCI,last_3F_time_diff,leg,pop,prize,error_code,father,mother,broodmare_sire,broodmare_sire_type,horse_color,id,id_for_fold,field_type,flat_or_jump,turn_type,race_type,waku,datetime,age_type,dist_type,is_1st_rank,is_in_3rd_rank,target,horse_Glicko2,horse_Glicko2_RD,horse_Glicko2_min,horse_Glicko2_max
9264,202504200603081110,2025,4,20,3,中山,8,11,クロワデュノール,1102,1151,サンデーレーシング,ノーザンファーム,安平町,18,5,10,195,17,4,2000,良,曇,12,牡,3,57.0,NaN,500.0,4.0,3,1.5,1.1,1.1,2,0.3,1573.0,4.0,4.0,6.0,2.0,34.7,8,35.4,52.0,53.87,51.2,0.4,先行,1.0,8000,0,キタサンブラック,ライジングクロス,Cape Cross,ニアークティック系,青鹿,2022105102,2025042006030811,芝,平地,R,中山芝2000,outer,2025-04-20 11:02:00,3,intermediate,0,1,2,2143.086071,67.219076,1941.428842,2344.7433
3583,202506010502121113,2025,6,1,2,東京,C,11,クロワデュノール,1102,1151,サンデーレーシング,ノーザンファーム,安平町,18,7,13,195,11,4,2400,良,晴,12,牡,3,57.0,NaN,504.0,4.0,3,2.1,1.2,1.1,1,-0.1,2237.0,4.0,3.0,2.0,3.0,34.2,4,36.5,56.7,57.40,54.3,0.6,先行,1.0,30000,0,キタサンブラック,ライジングクロス,Cape Cross,ニアークティック系,青鹿,2022105102,2025060105021211,芝,平地,L,東京芝2400,outer,2025-06-01 11:01:00,3,long,1,1,1,2143.086071,67.219076,1941.428842,2344.7433


In [13]:
print(feature_col_pred)

['horse_Glicko2', 'horse_Glicko2_RD', 'horse_Glicko2_min', 'horse_Glicko2_max']


# 大丈夫そう